In [1]:
import os
import copy
import json
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [13]:
job_id = "3dcgyav" # os.environ["job_id"]
data_path = f"/code/data/{job_id}"

with open(os.path.join(data_path, "data.json"), "r", encoding="UTF-8") as f:
    data = json.load(f)

with open(os.path.join(data_path, "config.json"), "r", encoding="UTF-8") as c:
    config = json.load(c)

In [14]:
df = pd.read_json(data, orient="key")
df["date"] = pd.to_datetime(df["date"])

In [15]:
df.head()

,date,last_price,start_price,high_price,low_price,volume
0,2021-01-04,83000,81000,84400,80200,38655276
1,2021-01-05,83900,81600,83900,81600,35335669
2,2021-01-06,82200,83300,84500,82100,42089013
3,2021-01-07,82900,82800,84200,82700,32644642
4,2021-01-08,88800,83300,90000,83000,59013307


In [16]:
config

{'model_name': 'TFT',
 'valid_rate': 0.8,
 'input_window': 3,
 'output_window': 1,
 'job_id': '3dcgyav'}